1. Import the LLM
2. Import Tool
3. Bind tool to LLM
4. Prompt Template
5. Create a Custom Agent

## Import LLM

In [3]:
from langchain_openai import ChatOpenAI
import os

llm = ChatOpenAI(model="gpt-3.5-turbo", 
                 temperature=0, 
                 openai_api_key=os.environ.get("OPENAI_API_KEY"))

In [2]:
llm.invoke("Who won the match between RCB vs CSK on 18th may 2024?")

AIMessage(content="I'm sorry, but I cannot provide real-time information as I do not have access to the internet. Please check the latest sports news or visit a reliable sports website for the most up-to-date information on the match between RCB and CSK on 18th May 2024.", response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 26, 'total_tokens': 84}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-257b8c2d-910d-40e4-a726-6e264d7f2895-0')

In [3]:
llm.invoke("Which two teams are in the finals of UEFA champions league 2024?").content

"I'm sorry, but as an AI, I do not have real-time information. Please check the latest updates on the UEFA Champions League 2024 to find out which two teams are in the finals."

## Import Tool

In [4]:
# !pip install google-api-python-client
# !pip install -U langchain-google-community

In [1]:
from langchain_google_community import GoogleSearchRun, GoogleSearchAPIWrapper

In [2]:
wrapper = GoogleSearchAPIWrapper(google_api_key=os.environ.get("GOOGLE_API_KEY"), google_cse_id = os.environ.get("GOOGLE_CSE_ID"))
google_search = GoogleSearchRun(api_wrapper = wrapper)

In [5]:
# print(google_search.invoke("Who won the match between RCB vs CSK on 18th may 2024?"))

In [8]:
print(google_search.invoke("site:cricinfo.com Who won the match between RCB vs CSK on 18th may 2024?"))

Live Cricket Scoreboard: Get Royal Challengers Bengaluru vs Chennai Super Kings 68th Match, cricket scorecard, Indian Premier League 2024 dated May 18, ... Debut/Last Matches of Yash Dayal ; Debut. Maharashtra vs UP at Delhi- February 21, 2019 ; Last. RCB vs CSK at Bengaluru- May 18, 2024. Find out where and when 2024 IPL matches are being played and follow along for the latest updates. Indian Premier League 2024 Matches. RCB vs CSK - 1st Match|DC ... - He was the Player of the Match in CSK's thrilling win ... Timeout crew on CSK's clever use of bowling partnerships. May 04, 2024 ... of their seats. RCB vs CSK ... His tidy bowling helped RCB win their last six matches to qualify for the playoffs. ... 17-May-2024, Bengaluru ... RCB vs CSK at Bengaluru- May 18, 2024 · View more ... Recent Matches of Virat Kohli ; RCB vs CSK, 47, 17-May-2024, Bengaluru · T20 ; RCB vs DC, 27, 11-May-2024, Bengaluru · T20. Recent Matches of Dinesh Karthik. Match, Bat, Wkt, Date, Ground, Format. RCB vs CSK, 

## Bind tool to LLM

In [9]:
tools = [google_search]

In [10]:
llm_with_tools = llm.bind_tools(tools)

## Prompt Template

In [11]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, designed to answer the current events with the help of provided tools.",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

## Create Custom Agent

In [12]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [13]:
from langchain.callbacks import get_openai_callback

In [14]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, stream_runnable = False)

In [15]:
with get_openai_callback() as cb:
    out = agent_executor.invoke({"input": "Who won the match between RCB vs CSK on 18th may 2024?"})
print(out)
print(cb)



> Entering new AgentExecutor chain...

Invoking: `google_search` with `RCB vs CSK 18th May 2024 match result`


4 days ago ... BENGALURU WIN BY 27 RUNS AND MAKES IT TO THE PLAYOFFS! 0. 19.5. Yash Dayal to Ravindra Jadeja, no run. Redemption for Yash ... 4 days ago ... RCB Vs CSK Live Score: RCB post 218/5, need to restrict CSK below 200 to qualify for playoffs. RCB Vs CSK Live Score: In a do-or-die match, ... 3 days ago ... RCB vs CSK Live Score, IPL 2024: Bengaluru 218/5 vs Chennai Super Kings. Glenn Maxwell came in and he did his part as well in that final over. 4 days ago ... All of it summed up the drama of the night in Bengaluru, as Royal Challengers Bengaluru (RCB) won their sixth game on the trot in IPL 2024 to ... 4 days ago ... Rcb vs csk p1 annexe one physical ticket available at 16k dm for details. Upvote 4 days ago ... Aaarrrr Ceeeee Beeeeee in IPL 2024 playoffs. Yash Dayal bowled a superb final over, Royal Challengers Bengaluru vs Chennai Super Kings,. 4 days ago ... Thi

In [16]:
with get_openai_callback() as cb:
    out = agent_executor.invoke({"input": "Which two teams are in the finals of UEFA champions league 2024?"})
print(out)
print(cb)



> Entering new AgentExecutor chain...

Invoking: `google_search` with `UEFA Champions League 2024 finals teams`


May 10, 2024 ... Next season's final will take place at the Fussball Arena München in Munich, Germany, on Saturday 31 May. © 1998-2024 UEFA. All rights reserved. It will be held at Wembley Stadium in London, England, on 1 June 2024, between German club Borussia Dortmund and Spanish club Real Madrid. Due to the ... 2 days ago ... Domestic qualifiers as it stands ; England (4) · Man City · Arsenal ; Spain (4) · Real Madrid · Barcelona ; Germany (4) · Leverkusen · Stuttgart ; Italy ( ... They will also qualify for the final of the brand-new 2024 FIFA Intercontinental Cup and the expanded 2025 FIFA Club World Cup in the United States. This is the ... Apr 16, 2024 ... UEFA Champions League quarterfinal draw 2024 in full ; Arsenal vs Bayern Munich, Arsenal 2-2 Bayern, April 17 ; Atletico Madrid 4-5 Borussia ... Mar 12, 2024 ... Quarterfinal, semifinal, final dates. The Champion